In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
post = pd.read_csv('lv2-2402.csv')

# 1

In [3]:
df1 = post.copy()

# 1-1

In [6]:
df1[df1['LIKES'] == 'yes']['LIKE_TYPE'].unique()

array(['heart emoji', 'laughing', nan, 'thumbs up', 'fire', 'clap'],
      dtype=object)

In [8]:
df1[df1['LIKES'] == 'no']['LIKE_TYPE'].unique()

array([nan, 'fire', 'laughing', 'clap', 'heart emoji', 'thumbs up'],
      dtype=object)

In [14]:
yes_map = {
    'heart emoji': 5,
    'thumbs up': 4,
    'clap': 3,
    'laughing': 2,
    'fire': 0
}

df1.loc[df1['LIKES'] == 'yes', 'LIKE_SCORE'] = df1.loc[df1['LIKES'] == 'yes', 'LIKE_TYPE'].map(yes_map).fillna(value=1)

In [16]:
no_map = {
    'fire': -5, 
    'laughing': 0,
    'clap': 0,
    'heart emoji': 0,
    'thumbs up': 0
}

df1.loc[df1['LIKES'] == 'no', 'LIKE_SCORE'] = df1.loc[df1['LIKES'] == 'no', 'LIKE_TYPE'].map(no_map).fillna(value=-1)

## 1-2

In [20]:
AGG24 = df1.groupby('POST_ID').apply(lambda x: pd.Series({
    'NUM_LIKES': (x['LIKES'] == 'yes').sum(),
    'NUM_USER': x['USER_ID'].nunique(),
    'SUM_SCORE': x['LIKE_SCORE'].sum()
}))

In [21]:
AGG24.shape

(257, 3)

## 1-3

In [22]:
AGG24['POPULAR'] = AGG24['NUM_LIKES'] + AGG24['SUM_SCORE'] / AGG24['NUM_USER']

In [26]:
AGG24['POPULAR'].max().round(2)

20.82

# 2

In [93]:
df2 = post.copy()

## 2-1

In [94]:
AGG25 = df2.groupby('USER_ID').apply(lambda x: pd.Series({
    'NUM_POST': x['POST_ID'].nunique(),
    'NUM_LIKE': x[x['LIKES'] == 'yes']['POST_ID'].nunique()
}))

In [95]:
AGG25.shape

(77, 2)

## 2-2

In [96]:
from sklearn.preprocessing import StandardScaler

In [97]:
df_scaled = pd.DataFrame(StandardScaler().fit_transform(AGG25), columns=['NUM_POST_S', 'NUM_LIKE_S'])

In [98]:
AGG25 = pd.concat([AGG25.reset_index(), df_scaled], axis=1)

In [99]:
# AGG25[['NUM_POST_S', 'NUM_LIKE_S']] = pd.DataFrame(StandardScaler().fit_transform(AGG25),
#                                                    columns=['NUM_POST_S', 'NUM_LIKE_S'],\
#                                                    index=AGG25.index)

In [100]:
AGG25

,USER_ID,NUM_POST,NUM_LIKE,NUM_POST_S,NUM_LIKE_S
0,2,66,29,-0.432418,-0.525662
1,3,67,25,-0.418579,-0.632813
2,4,64,32,-0.460095,-0.445300
3,5,257,135,2.210794,2.313819
4,6,62,31,-0.487773,-0.472087
...,...,...,...,...,...
72,96,75,42,-0.307869,-0.177424
73,97,68,41,-0.404740,-0.204212
74,98,65,28,-0.446257,-0.552450
75,99,68,39,-0.404740,-0.257787


## 2-3

In [101]:
from sklearn.cluster import KMeans

In [104]:
col = ['NUM_POST_S', 'NUM_LIKE_S']

In [103]:
model = KMeans(n_clusters=2, random_state=1234).fit(AGG25[col])

## 2-4

In [107]:
AGG25['cluster'] = model.predict(AGG25[col])

In [110]:
Cluster1 = AGG25['cluster'].value_counts().idxmin()
Cluster2 = AGG25['cluster'].value_counts().idxmax()

## 2-5

In [112]:
AGG25

,USER_ID,NUM_POST,NUM_LIKE,NUM_POST_S,NUM_LIKE_S,cluster
0,2,66,29,-0.432418,-0.525662,0
1,3,67,25,-0.418579,-0.632813,0
2,4,64,32,-0.460095,-0.445300,0
3,5,257,135,2.210794,2.313819,1
4,6,62,31,-0.487773,-0.472087,0
...,...,...,...,...,...,...
72,96,75,42,-0.307869,-0.177424,0
73,97,68,41,-0.404740,-0.204212,0
74,98,65,28,-0.446257,-0.552450,0
75,99,68,39,-0.404740,-0.257787,0


In [114]:
df2 = pd.merge(df2, AGG25, on='USER_ID', how='left')

In [118]:
Avg_A = df2[df2['cluster'] == Cluster1]['NUM_HASHTAG'].mean()
Avg_A

2.402274768033523

## 2-6

In [122]:
Avg_B = df2[df2['cluster'] == Cluster2]['NUM_HASHTAG'].mean()
Avg_B

2.428261393778635

In [124]:
round(abs(Avg_A - Avg_B), 3)

0.026

# 3

In [125]:
df3 = post.copy()

## 3-1

In [128]:
AGG26 = df3.groupby(['POST_ID', 'CREATED_DATE']).apply(lambda x: pd.Series({
    'NUM_USER': x['USER_ID'].nunique(),
    'NUM_LIKES': (x['LIKES'] == 'yes').sum(),
    'AVG_HASHTAG': x['NUM_HASHTAG'].mean()
}))

In [129]:
AGG26.shape

(257, 3)

## 3-2

In [132]:
AGG26['POPULAR'] = np.where(AGG26['NUM_USER'] >= 31, 1, 0)

## 3-3

In [143]:
from datetime import datetime

In [135]:
AGG26 = AGG26.reset_index()

In [145]:
AGG26['WEEKDAY'] = AGG26['CREATED_DATE'].map(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w'))

## 3-4

In [146]:
DataSet6 = AGG26.copy()

In [147]:
from sklearn.naive_bayes import GaussianNB

In [148]:
col = ['NUM_LIKES', 'AVG_HASHTAG', 'WEEKDAY']

In [149]:
model = GaussianNB().fit(X=DataSet6[col], y=DataSet6['POPULAR'])

## 3-5

In [152]:
TestSet = pd.DataFrame([{'NUM_LIKES': 16,
                         'AVG_HASHTAG': 2.42,
                         'WEEKDAY': 5}])
TestSet

,NUM_LIKES,AVG_HASHTAG,WEEKDAY
0,16,2.42,5


In [157]:
proba = model.predict_proba(TestSet[col])

In [158]:
proba[:, 1]

array([0.77612467])

In [159]:
round(0.77612467, 3)

0.776